<a href="https://colab.research.google.com/github/testproba1234/product-title-classifier/blob/main/colab/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Uvoz biblioteka

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Učitavanje podataka

In [ ]:
file_path = "https://raw.githubusercontent.com/testproba1234/product-title-classifier/main/data/products.csv"

df = pd.read_csv(file_path)
print("✅ Podaci uspešno učitani!")
print(df.head())


✅ Podaci uspešno učitani!
   product ID                                      Product Title  Merchant ID  \
0           1                    apple iphone 8 plus 64gb silver            1   
1           2                apple iphone 8 plus 64 gb spacegrau            2   
2           3  apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...            3   
3           4                apple iphone 8 plus 64gb space grey            4   
4           5  apple iphone 8 plus gold 5.5 64gb 4g unlocked ...            5   

   Category Label _Product Code  Number_of_Views  Merchant Rating  \
0   Mobile Phones    QA-2276-XC            860.0              2.5   
1   Mobile Phones    KA-2501-QO           3772.0              4.8   
2   Mobile Phones    FP-8086-IE           3092.0              3.9   
3   Mobile Phones    YI-0086-US            466.0              3.4   
4   Mobile Phones    NZ-3586-WP           4426.0              1.6   

   Listing Date    
0       5/10/2024  
1      12/31/2024  
2      11/10

# 3. Priprema podataka

In [ ]:
from sklearn.model_selection import train_test_split

# Ako ima više klasa nego redova u test skupu — isključi stratify
test_size = 0.2
n_test = int(len(df) * test_size)
n_classes = df['category'].nunique()

if n_classes > n_test:
    print(f"Previše klasa ({n_classes}) u odnosu na veličinu test skupa ({n_test}). Stratify se isključuje.")
    stratify_param = None
else:
    stratify_param = df['category']

# Podela na trening i test skup bez greške
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=42, stratify=stratify_param
)

print(f"Podela uspešna!")
print(f"Trening skup: {len(X_train)} primera")
print(f"Test skup: {len(X_test)} primera")

⚠️ Previše klasa (3110) u odnosu na veličinu test skupa (1463). Stratify se isključuje.
✅ Podela uspešna!
Trening skup: 5854 primera
Test skup: 1464 primera


# 4. Vektorizacija teksta

In [ ]:
# 🔧 Osiguraj da su svi naslovi stringovi i bez praznih vrednosti
X_train = X_train.astype(str).fillna("")
X_test = X_test.astype(str).fillna("")

# Sada vektorizacija može bez greške
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english", max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("TF-IDF vektorizacija uspešna!")
print("Dimenzije trening skupa:", X_train_tfidf.shape)
print("Dimenzije test skupa:", X_test_tfidf.shape)

TF-IDF vektorizacija uspešna!
Dimenzije trening skupa: (5854, 5000)
Dimenzije test skupa: (1464, 5000)


# 5. Definisanje modela

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Linear SVM": LinearSVC()
}

# 6. Trening i evaluacija

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

# Definiši sve modele koje želiš testirati
models = {
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "Multinomial Naive Bayes": MultinomialNB(),
    "Linear SVC": LinearSVC(random_state=42)
}

print("✅ Modeli uspešno definisani:", list(models.keys()))

results = {}

for name, model in models.items():
    print(f"\nTreniranje modela: {name}")
    model.fit(X_train_tfidf, y_train)
    preds = model.predict(X_test_tfidf)

    acc = accuracy_score(y_test, preds)
    results[name] = acc
    print(f"Tačnost: {acc:.4f}")
    print("Ended")

✅ Modeli uspešno definisani: ['Logistic Regression', 'Multinomial Naive Bayes', 'Linear SVC']

Treniranje modela: Logistic Regression


NameError: name 'X_train_tfidf' is not defined

# 7. Vizualizacija rezultata

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(x=list(results.keys()), y=list(results.values()))
plt.title("Uporedna tačnost modela")
plt.ylabel("Accuracy")
plt.ylim(0,1)
plt.show()

# 8. Izbor najboljeg modela

In [ ]:
best_model_name = max(results, key=results.get)
best_model = models[best_model_name]
print(f"\n🏆 Najbolji model: {best_model_name} sa tačnošću od {results[best_model_name]:.4f}")

# 9. Čuvanje modela i vektorizatora

In [ ]:
import joblib

joblib.dump(best_model, "best_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

print("Model i vektorizator uspešno sačuvani!")